In [1]:
# NOTE: THE REDUNDANT IMPORTS AND FUNCTION DEFINITIONS IS INTENTIONAL TO MAKE THE CODE CELLS SELF CONTAINED 
import json
from logging import raiseExceptions 
import tweepy
import time
from datetime import datetime
import time
import os

# PRINT TWEEPY VERSION
print("tweepy version =",tweepy.__version__)

#----------------------
# READ API KEY FILE
#----------------------
f = open("/Users/xiaocheng/Downloads/api-keys.json")
input=json.load(f); #
#print(input)

# LOAD KEYS INTO API
consumer_key=input["consumer_key"]    
consumer_secret=input["consumer_secret"]    
access_token=input["access_token"]    
access_token_secret=input["access_token_secret"]    
bearer_token=input["bearer_token"]   

#----------------------
# DEFINE USEFUL FUNCTIONS
#----------------------

# DEFINE PRETTY PRINT JSON FUNCTION
def pretty_print_json(input):
    print(json.dumps(input, indent=4))

# DEFINE FUNCTION TO SAVE TWEEPY SEARCH RESULTS
#   searches=array with various tweepy search objects
#   TODO: ADD "full and sparse" mode
#          full = save all tweet data (100 tweeks ~ 1 MB  --> 100,000 ~ 1 GB)
#          sparse = only save most important info
def save_search_tweets_results(searches,info_str="",output_name="tweet-search.json"):
    # if(str(type(input)) == "<class 'tweepy.models.SearchResults'>"):
    if(str(type(searches)) == "<class 'list'>"):
        #COMBINE ALL JSONS FOR VARIOUS TWEETS INTO ON BIG JSON CALL "out"
        out={}
        out["search_info"]=info_str

        #LOOP OVER SEARCHES
        tweet_ids=[]
        k=0 #counter
        for search in searches:
            #LOOP OVER TWEETS IN SEARCH
            for i in range(0,len(search)):
                out[str(k)]=search[i]._json
                tweet_id=search[i]._json["id_str"]
                #CHECK FOR REDUNDANT TWEETS
                if tweet_id in tweet_ids:
                    print("WARNING: REPEATED TWEETS IN SAVED FILE; ID = ",tweet_id)
                tweet_ids.append(search[i]._json["id_str"])

                k+=1
            #pretty_print_json(out)

        #DELETE FILE IF IT EXIST (START FRESH)
        if os.path.exists(output_name):
            os.remove(output_name)

        #WRITE FILE
        with open(output_name, 'w') as f:
            json.dump(out, f)
    else: 
        raise RuntimeError("ERROR: Incorrect datatype")

#----------------------
# SET UP CONNECTION
#----------------------
#   Use special options when initializing the API. These tell
#   it to wait while the Twitter time-limit windows elapse
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True) 

#----------------------
# RUN SEARCH 
#----------------------

#SEARCH PARAM
query="new graduate"

# NUMBER OF TWEETS TO SEARCH 
# number_of_tweets=1000 
number_of_tweets=6000
# ideally use multiples of 100 for number_of_tweets
# should be able to collect 18000 tweets every 15 minutes
start_time = time.time()
max_loop_time_hrs=5

# THIS WILL KEEP DOING SEARCHES FURTHER AND FURTHER BACK IN TIME
# USING THE MAX_ID TO THE TIMELINE 
num_tweets_collected=0
searches=[]
k=0
#KEEP SEARCHING UNTIL DESIRED NUMBER OF TWEETS COLLECTED
while num_tweets_collected<number_of_tweets or (time.time()-start_time)/60./60>max_loop_time_hrs: 
    try: 
        #FIRST SEARCH
        if len(searches)==0:
            search_results = api.search_tweets(query, lang="en", count=100)
        #ADDITIONAL SEARCHES
        else:
            search_results = api.search_tweets(query, lang="en", count=100,max_id=max_id_next)

        #UPDATE PARAMETERS
        num_tweets_collected+=len(search_results)
        max_id_next=int(search_results[-1]._json["id_str"])-1

        #SAVE SEARCH RESULTS
        searches.append(search_results)

        #SAVE TEMPORARY CHECKPOINTS (DONT DO TOO OFTEN .. SLOWS CODE DOWN)
        if(k%10==0):
            print("SEARCH-"+str(k)+" COMPLETED;  TWEETS_COLLECTED=",num_tweets_collected,"; TIME (s) = ",time.time() - start_time)
        if(k%25==0):
            save_search_tweets_results(searches,output_name="tmp-snapshot.json")
            
        k+=1
    except:
        print("WARNING: twitter search failed")

    #SLEEP 5 SECONDS BEFORE NEXT REQUEST 
    if(number_of_tweets>18000):
        time.sleep(5)
    else:
        time.sleep(0.2)
        

# REPORT BASIC SEARCH INFO
print(num_tweets_collected,len(searches))
print("search time (s) =", (time.time() - start_time)) #/60.)

#TIMESTAMP SEARCH 
now = datetime.now()
dt_string = now.strftime("%Y-%m-%Y-H%H-M%M-S%S")

#----------------------
# SAVE RESULTS
#----------------------
info_str="query = "+query+"; number_of_tweets = "+str(number_of_tweets)+"; date = "+str(dt_string)
out_name=str(dt_string)+"-twitter-search.json"
save_search_tweets_results(searches,info_str=info_str,output_name=out_name)

#CLEAN-UP TEMP FILES
os.remove("tmp-snapshot.json")
# import glob
# list_to_delete=glob.glob("./*-tmp-snapshot.json")
# for file in list_to_delete:
#     os.remove(file)

Rate limit reached. Sleeping for: 601


tweepy version = 4.12.1
SEARCH-0 COMPLETED;  TWEETS_COLLECTED= 100 ; TIME (s) =  602.8242211341858
SEARCH-10 COMPLETED;  TWEETS_COLLECTED= 1100 ; TIME (s) =  609.6143691539764
SEARCH-20 COMPLETED;  TWEETS_COLLECTED= 2100 ; TIME (s) =  616.7975571155548
SEARCH-30 COMPLETED;  TWEETS_COLLECTED= 3100 ; TIME (s) =  624.7018320560455
SEARCH-40 COMPLETED;  TWEETS_COLLECTED= 4100 ; TIME (s) =  636.3923442363739
SEARCH-50 COMPLETED;  TWEETS_COLLECTED= 5100 ; TIME (s) =  644.432225227356
6000 60
search time (s) = 655.1603240966797


In [4]:
import json
import pandas as pd
with open('2022-12-2022-H23-M52-S35-twitter-search.json', 'r') as f:
    data = json.load(f)
df = pd.DataFrame(data).T
datadf = df.drop("search_info")
datadf.to_csv('raw_data1.csv')
import pandas as pd
r = pd.read_csv("raw_data1.csv")

In [1]:
# NOTE: THE REDUNDANT IMPORTS AND FUNCTION DEFINITIONS IS INTENTIONAL TO MAKE THE CODE CELLS SELF CONTAINED 
import json
from logging import raiseExceptions 
import tweepy
import time
from datetime import datetime
import time
import os

# PRINT TWEEPY VERSION
print("tweepy version =",tweepy.__version__)

#----------------------
# READ API KEY FILE
#----------------------
f = open("/Users/xiaocheng/Downloads/api-keys.json")
input=json.load(f); #
#print(input)

# LOAD KEYS INTO API
consumer_key=input["consumer_key"]    
consumer_secret=input["consumer_secret"]    
access_token=input["access_token"]    
access_token_secret=input["access_token_secret"]    
bearer_token=input["bearer_token"]   

#----------------------
# DEFINE USEFUL FUNCTIONS
#----------------------

# DEFINE PRETTY PRINT JSON FUNCTION
def pretty_print_json(input):
    print(json.dumps(input, indent=4))

# DEFINE FUNCTION TO SAVE TWEEPY SEARCH RESULTS
#   searches=array with various tweepy search objects
#   TODO: ADD "full and sparse" mode
#          full = save all tweet data (100 tweeks ~ 1 MB  --> 100,000 ~ 1 GB)
#          sparse = only save most important info
def save_search_tweets_results(searches,info_str="",output_name="tweet-search.json"):
    # if(str(type(input)) == "<class 'tweepy.models.SearchResults'>"):
    if(str(type(searches)) == "<class 'list'>"):
        #COMBINE ALL JSONS FOR VARIOUS TWEETS INTO ON BIG JSON CALL "out"
        out={}
        out["search_info"]=info_str

        #LOOP OVER SEARCHES
        tweet_ids=[]
        k=0 #counter
        for search in searches:
            #LOOP OVER TWEETS IN SEARCH
            for i in range(0,len(search)):
                out[str(k)]=search[i]._json
                tweet_id=search[i]._json["id_str"]
                #CHECK FOR REDUNDANT TWEETS
                if tweet_id in tweet_ids:
                    print("WARNING: REPEATED TWEETS IN SAVED FILE; ID = ",tweet_id)
                tweet_ids.append(search[i]._json["id_str"])

                k+=1
            #pretty_print_json(out)

        #DELETE FILE IF IT EXIST (START FRESH)
        if os.path.exists(output_name):
            os.remove(output_name)

        #WRITE FILE
        with open(output_name, 'w') as f:
            json.dump(out, f)
    else: 
        raise RuntimeError("ERROR: Incorrect datatype")

#----------------------
# SET UP CONNECTION
#----------------------
#   Use special options when initializing the API. These tell
#   it to wait while the Twitter time-limit windows elapse
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True) 

#----------------------
# RUN SEARCH 
#----------------------

#SEARCH PARAM
query="job hunting"

# NUMBER OF TWEETS TO SEARCH 
# number_of_tweets=1000 
number_of_tweets=6000
# ideally use multiples of 100 for number_of_tweets
# should be able to collect 18000 tweets every 15 minutes
start_time = time.time()
max_loop_time_hrs=5

# THIS WILL KEEP DOING SEARCHES FURTHER AND FURTHER BACK IN TIME
# USING THE MAX_ID TO THE TIMELINE 
num_tweets_collected=0
searches=[]
k=0
#KEEP SEARCHING UNTIL DESIRED NUMBER OF TWEETS COLLECTED
while num_tweets_collected<number_of_tweets or (time.time()-start_time)/60./60>max_loop_time_hrs: 
    try: 
        #FIRST SEARCH
        if len(searches)==0:
            search_results = api.search_tweets(query, lang="en", count=100)
        #ADDITIONAL SEARCHES
        else:
            search_results = api.search_tweets(query, lang="en", count=100,max_id=max_id_next)

        #UPDATE PARAMETERS
        num_tweets_collected+=len(search_results)
        max_id_next=int(search_results[-1]._json["id_str"])-1

        #SAVE SEARCH RESULTS
        searches.append(search_results)

        #SAVE TEMPORARY CHECKPOINTS (DONT DO TOO OFTEN .. SLOWS CODE DOWN)
        if(k%10==0):
            print("SEARCH-"+str(k)+" COMPLETED;  TWEETS_COLLECTED=",num_tweets_collected,"; TIME (s) = ",time.time() - start_time)
        if(k%25==0):
            save_search_tweets_results(searches,output_name="tmp-snapshot.json")
            
        k+=1
    except:
        print("WARNING: twitter search failed")

    #SLEEP 5 SECONDS BEFORE NEXT REQUEST 
    if(number_of_tweets>18000):
        time.sleep(5)
    else:
        time.sleep(0.2)
        

# REPORT BASIC SEARCH INFO
print(num_tweets_collected,len(searches))
print("search time (s) =", (time.time() - start_time)) #/60.)

#TIMESTAMP SEARCH 
now = datetime.now()
dt_string = now.strftime("%Y-%m-%Y-H%H-M%M-S%S")

#----------------------
# SAVE RESULTS
#----------------------
info_str="query = "+query+"; number_of_tweets = "+str(number_of_tweets)+"; date = "+str(dt_string)
out_name=str(dt_string)+"-twitter-search.json"
save_search_tweets_results(searches,info_str=info_str,output_name=out_name)

#CLEAN-UP TEMP FILES
os.remove("tmp-snapshot.json")
# import glob
# list_to_delete=glob.glob("./*-tmp-snapshot.json")
# for file in list_to_delete:
#     os.remove(file)

tweepy version = 4.12.1
SEARCH-0 COMPLETED;  TWEETS_COLLECTED= 86 ; TIME (s) =  0.4633622169494629
SEARCH-10 COMPLETED;  TWEETS_COLLECTED= 1031 ; TIME (s) =  7.978933095932007
SEARCH-20 COMPLETED;  TWEETS_COLLECTED= 1761 ; TIME (s) =  14.799990177154541
SEARCH-30 COMPLETED;  TWEETS_COLLECTED= 2752 ; TIME (s) =  22.47602605819702
SEARCH-40 COMPLETED;  TWEETS_COLLECTED= 3752 ; TIME (s) =  30.809023141860962
SEARCH-50 COMPLETED;  TWEETS_COLLECTED= 4752 ; TIME (s) =  40.17376399040222
SEARCH-60 COMPLETED;  TWEETS_COLLECTED= 5752 ; TIME (s) =  49.970667123794556
6052 64
search time (s) = 52.854995012283325


In [2]:
import json
import pandas as pd
with open('2022-12-2022-H11-M30-S28-twitter-search.json', 'r') as f:
    data = json.load(f)
df = pd.DataFrame(data).T
datadf = df.drop("search_info")
datadf.to_csv('raw_data2.csv')
import pandas as pd
r = pd.read_csv("raw_data2.csv")

In [3]:
# NOTE: THE REDUNDANT IMPORTS AND FUNCTION DEFINITIONS IS INTENTIONAL TO MAKE THE CODE CELLS SELF CONTAINED 
import json
from logging import raiseExceptions 
import tweepy
import time
from datetime import datetime
import time
import os

# PRINT TWEEPY VERSION
print("tweepy version =",tweepy.__version__)

#----------------------
# READ API KEY FILE
#----------------------
f = open("/Users/xiaocheng/Downloads/api-keys.json")
input=json.load(f); #
#print(input)

# LOAD KEYS INTO API
consumer_key=input["consumer_key"]    
consumer_secret=input["consumer_secret"]    
access_token=input["access_token"]    
access_token_secret=input["access_token_secret"]    
bearer_token=input["bearer_token"]   

#----------------------
# DEFINE USEFUL FUNCTIONS
#----------------------

# DEFINE PRETTY PRINT JSON FUNCTION
def pretty_print_json(input):
    print(json.dumps(input, indent=4))

# DEFINE FUNCTION TO SAVE TWEEPY SEARCH RESULTS
#   searches=array with various tweepy search objects
#   TODO: ADD "full and sparse" mode
#          full = save all tweet data (100 tweeks ~ 1 MB  --> 100,000 ~ 1 GB)
#          sparse = only save most important info
def save_search_tweets_results(searches,info_str="",output_name="tweet-search.json"):
    # if(str(type(input)) == "<class 'tweepy.models.SearchResults'>"):
    if(str(type(searches)) == "<class 'list'>"):
        #COMBINE ALL JSONS FOR VARIOUS TWEETS INTO ON BIG JSON CALL "out"
        out={}
        out["search_info"]=info_str

        #LOOP OVER SEARCHES
        tweet_ids=[]
        k=0 #counter
        for search in searches:
            #LOOP OVER TWEETS IN SEARCH
            for i in range(0,len(search)):
                out[str(k)]=search[i]._json
                tweet_id=search[i]._json["id_str"]
                #CHECK FOR REDUNDANT TWEETS
                if tweet_id in tweet_ids:
                    print("WARNING: REPEATED TWEETS IN SAVED FILE; ID = ",tweet_id)
                tweet_ids.append(search[i]._json["id_str"])

                k+=1
            #pretty_print_json(out)

        #DELETE FILE IF IT EXIST (START FRESH)
        if os.path.exists(output_name):
            os.remove(output_name)

        #WRITE FILE
        with open(output_name, 'w') as f:
            json.dump(out, f)
    else: 
        raise RuntimeError("ERROR: Incorrect datatype")

#----------------------
# SET UP CONNECTION
#----------------------
#   Use special options when initializing the API. These tell
#   it to wait while the Twitter time-limit windows elapse
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True) 

#----------------------
# RUN SEARCH 
#----------------------

#SEARCH PARAM
query="job OR work"

# NUMBER OF TWEETS TO SEARCH 
# number_of_tweets=1000 
number_of_tweets=10000
# ideally use multiples of 100 for number_of_tweets
# should be able to collect 18000 tweets every 15 minutes
start_time = time.time()
max_loop_time_hrs=5

# THIS WILL KEEP DOING SEARCHES FURTHER AND FURTHER BACK IN TIME
# USING THE MAX_ID TO THE TIMELINE 
num_tweets_collected=0
searches=[]
k=0
#KEEP SEARCHING UNTIL DESIRED NUMBER OF TWEETS COLLECTED
while num_tweets_collected<number_of_tweets or (time.time()-start_time)/60./60>max_loop_time_hrs: 
    try: 
        #FIRST SEARCH
        if len(searches)==0:
            search_results = api.search_tweets(query, lang="en", count=100)
        #ADDITIONAL SEARCHES
        else:
            search_results = api.search_tweets(query, lang="en", count=100,max_id=max_id_next)

        #UPDATE PARAMETERS
        num_tweets_collected+=len(search_results)
        max_id_next=int(search_results[-1]._json["id_str"])-1

        #SAVE SEARCH RESULTS
        searches.append(search_results)

        #SAVE TEMPORARY CHECKPOINTS (DONT DO TOO OFTEN .. SLOWS CODE DOWN)
        if(k%10==0):
            print("SEARCH-"+str(k)+" COMPLETED;  TWEETS_COLLECTED=",num_tweets_collected,"; TIME (s) = ",time.time() - start_time)
        if(k%25==0):
            save_search_tweets_results(searches,output_name="tmp-snapshot.json")
            
        k+=1
    except:
        print("WARNING: twitter search failed")

    #SLEEP 5 SECONDS BEFORE NEXT REQUEST 
    if(number_of_tweets>18000):
        time.sleep(5)
    else:
        time.sleep(0.2)
        

# REPORT BASIC SEARCH INFO
print(num_tweets_collected,len(searches))
print("search time (s) =", (time.time() - start_time)) #/60.)

#TIMESTAMP SEARCH 
now = datetime.now()
dt_string = now.strftime("%Y-%m-%Y-H%H-M%M-S%S")

#----------------------
# SAVE RESULTS
#----------------------
info_str="query = "+query+"; number_of_tweets = "+str(number_of_tweets)+"; date = "+str(dt_string)
out_name=str(dt_string)+"-twitter-search.json"
save_search_tweets_results(searches,info_str=info_str,output_name=out_name)

#CLEAN-UP TEMP FILES
os.remove("tmp-snapshot.json")
# import glob
# list_to_delete=glob.glob("./*-tmp-snapshot.json")
# for file in list_to_delete:
#     os.remove(file)

tweepy version = 4.12.1
SEARCH-0 COMPLETED;  TWEETS_COLLECTED= 100 ; TIME (s) =  0.9849190711975098
SEARCH-10 COMPLETED;  TWEETS_COLLECTED= 1098 ; TIME (s) =  10.693579912185669
SEARCH-20 COMPLETED;  TWEETS_COLLECTED= 2092 ; TIME (s) =  19.351260900497437
SEARCH-30 COMPLETED;  TWEETS_COLLECTED= 3089 ; TIME (s) =  28.710843086242676
SEARCH-40 COMPLETED;  TWEETS_COLLECTED= 4084 ; TIME (s) =  37.510963916778564
SEARCH-50 COMPLETED;  TWEETS_COLLECTED= 5084 ; TIME (s) =  46.54157304763794
SEARCH-60 COMPLETED;  TWEETS_COLLECTED= 6065 ; TIME (s) =  56.35607385635376
SEARCH-70 COMPLETED;  TWEETS_COLLECTED= 7059 ; TIME (s) =  65.16458702087402
SEARCH-80 COMPLETED;  TWEETS_COLLECTED= 8056 ; TIME (s) =  76.18343114852905
SEARCH-90 COMPLETED;  TWEETS_COLLECTED= 9051 ; TIME (s) =  85.84900403022766
SEARCH-100 COMPLETED;  TWEETS_COLLECTED= 10045 ; TIME (s) =  94.60447406768799
10045 101
search time (s) = 97.55709314346313


In [4]:
import json
import pandas as pd
with open('2022-12-2022-H11-M34-S23-twitter-search.json', 'r') as f:
    data = json.load(f)
df = pd.DataFrame(data).T
datadf = df.drop("search_info")
datadf.to_csv('raw_data3.csv')
import pandas as pd
r = pd.read_csv("raw_data3.csv")

In [5]:
# NOTE: THE REDUNDANT IMPORTS AND FUNCTION DEFINITIONS IS INTENTIONAL TO MAKE THE CODE CELLS SELF CONTAINED 
import json
from logging import raiseExceptions 
import tweepy
import time
from datetime import datetime
import time
import os

# PRINT TWEEPY VERSION
print("tweepy version =",tweepy.__version__)

#----------------------
# READ API KEY FILE
#----------------------
f = open("/Users/xiaocheng/Downloads/api-keys.json")
input=json.load(f); #
#print(input)

# LOAD KEYS INTO API
consumer_key=input["consumer_key"]    
consumer_secret=input["consumer_secret"]    
access_token=input["access_token"]    
access_token_secret=input["access_token_secret"]    
bearer_token=input["bearer_token"]   

#----------------------
# DEFINE USEFUL FUNCTIONS
#----------------------

# DEFINE PRETTY PRINT JSON FUNCTION
def pretty_print_json(input):
    print(json.dumps(input, indent=4))

# DEFINE FUNCTION TO SAVE TWEEPY SEARCH RESULTS
#   searches=array with various tweepy search objects
#   TODO: ADD "full and sparse" mode
#          full = save all tweet data (100 tweeks ~ 1 MB  --> 100,000 ~ 1 GB)
#          sparse = only save most important info
def save_search_tweets_results(searches,info_str="",output_name="tweet-search.json"):
    # if(str(type(input)) == "<class 'tweepy.models.SearchResults'>"):
    if(str(type(searches)) == "<class 'list'>"):
        #COMBINE ALL JSONS FOR VARIOUS TWEETS INTO ON BIG JSON CALL "out"
        out={}
        out["search_info"]=info_str

        #LOOP OVER SEARCHES
        tweet_ids=[]
        k=0 #counter
        for search in searches:
            #LOOP OVER TWEETS IN SEARCH
            for i in range(0,len(search)):
                out[str(k)]=search[i]._json
                tweet_id=search[i]._json["id_str"]
                #CHECK FOR REDUNDANT TWEETS
                if tweet_id in tweet_ids:
                    print("WARNING: REPEATED TWEETS IN SAVED FILE; ID = ",tweet_id)
                tweet_ids.append(search[i]._json["id_str"])

                k+=1
            #pretty_print_json(out)

        #DELETE FILE IF IT EXIST (START FRESH)
        if os.path.exists(output_name):
            os.remove(output_name)

        #WRITE FILE
        with open(output_name, 'w') as f:
            json.dump(out, f)
    else: 
        raise RuntimeError("ERROR: Incorrect datatype")

#----------------------
# SET UP CONNECTION
#----------------------
#   Use special options when initializing the API. These tell
#   it to wait while the Twitter time-limit windows elapse
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True) 

#----------------------
# RUN SEARCH 
#----------------------

#SEARCH PARAM
query="retire"

# NUMBER OF TWEETS TO SEARCH 
# number_of_tweets=1000 
number_of_tweets=6000
# ideally use multiples of 100 for number_of_tweets
# should be able to collect 18000 tweets every 15 minutes
start_time = time.time()
max_loop_time_hrs=5

# THIS WILL KEEP DOING SEARCHES FURTHER AND FURTHER BACK IN TIME
# USING THE MAX_ID TO THE TIMELINE 
num_tweets_collected=0
searches=[]
k=0
#KEEP SEARCHING UNTIL DESIRED NUMBER OF TWEETS COLLECTED
while num_tweets_collected<number_of_tweets or (time.time()-start_time)/60./60>max_loop_time_hrs: 
    try: 
        #FIRST SEARCH
        if len(searches)==0:
            search_results = api.search_tweets(query, lang="en", count=100)
        #ADDITIONAL SEARCHES
        else:
            search_results = api.search_tweets(query, lang="en", count=100,max_id=max_id_next)

        #UPDATE PARAMETERS
        num_tweets_collected+=len(search_results)
        max_id_next=int(search_results[-1]._json["id_str"])-1

        #SAVE SEARCH RESULTS
        searches.append(search_results)

        #SAVE TEMPORARY CHECKPOINTS (DONT DO TOO OFTEN .. SLOWS CODE DOWN)
        if(k%10==0):
            print("SEARCH-"+str(k)+" COMPLETED;  TWEETS_COLLECTED=",num_tweets_collected,"; TIME (s) = ",time.time() - start_time)
        if(k%25==0):
            save_search_tweets_results(searches,output_name="tmp-snapshot.json")
            
        k+=1
    except:
        print("WARNING: twitter search failed")

    #SLEEP 5 SECONDS BEFORE NEXT REQUEST 
    if(number_of_tweets>18000):
        time.sleep(5)
    else:
        time.sleep(0.2)
        

# REPORT BASIC SEARCH INFO
print(num_tweets_collected,len(searches))
print("search time (s) =", (time.time() - start_time)) #/60.)

#TIMESTAMP SEARCH 
now = datetime.now()
dt_string = now.strftime("%Y-%m-%Y-H%H-M%M-S%S")

#----------------------
# SAVE RESULTS
#----------------------
info_str="query = "+query+"; number_of_tweets = "+str(number_of_tweets)+"; date = "+str(dt_string)
out_name=str(dt_string)+"-twitter-search.json"
save_search_tweets_results(searches,info_str=info_str,output_name=out_name)

#CLEAN-UP TEMP FILES
os.remove("tmp-snapshot.json")
# import glob
# list_to_delete=glob.glob("./*-tmp-snapshot.json")
# for file in list_to_delete:
#     os.remove(file)

tweepy version = 4.12.1
SEARCH-0 COMPLETED;  TWEETS_COLLECTED= 99 ; TIME (s) =  0.6936612129211426
SEARCH-10 COMPLETED;  TWEETS_COLLECTED= 1041 ; TIME (s) =  8.061005115509033
SEARCH-20 COMPLETED;  TWEETS_COLLECTED= 2032 ; TIME (s) =  15.304491996765137
SEARCH-30 COMPLETED;  TWEETS_COLLECTED= 3032 ; TIME (s) =  23.20733904838562
SEARCH-40 COMPLETED;  TWEETS_COLLECTED= 4031 ; TIME (s) =  30.49046230316162
SEARCH-50 COMPLETED;  TWEETS_COLLECTED= 5031 ; TIME (s) =  38.617154121398926
SEARCH-60 COMPLETED;  TWEETS_COLLECTED= 6026 ; TIME (s) =  47.20142889022827
6026 61
search time (s) = 47.4037811756134


In [6]:
import json
import pandas as pd
with open('2022-12-2022-H11-M47-S41-twitter-search.json', 'r') as f:
    data = json.load(f)
df = pd.DataFrame(data).T
datadf = df.drop("search_info")
datadf.to_csv('raw_data4.csv')
import pandas as pd
r = pd.read_csv("raw_data4.csv")